In [2]:
pip install datasets transformers rouge-score nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 102.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.

In [43]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [44]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [3]:
import transformers

print(transformers.__version__)

4.27.4


# Fine-tuning a model on a summarization task

In this notebook, we will see how to fine-tune one of the 🤗 Transformers model for a summarization task. We will use the XSum dataset (for extreme summarization) which contains BBC articles accompanied with single-sentence summaries.

In [ ]:
model_checkpoint = "t5-small"

## Loading the dataset

In [4]:
from google.colab import drive

drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks
#! ls

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [5]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('sample_10k_data_for_visualization.csv').sample(frac=1)
df_train = df.iloc[:9000]
df_val = df.iloc[9000:]
train_dataset = Dataset.from_pandas(df_train)
val_dataset = Dataset.from_pandas(df_val)

In [9]:
# from datasets import load_dataset, load_metric

# raw_datasets = load_dataset("xsum")
# metric = load_metric("rouge")

In [6]:
df_train.head()

,abstract,article,section_names
179,"b' counting the frequencies of 3- , 4- , and 5...",b'design tools for counting the frequencies of...,b'introduction\nproblem formulation\nprelimina...
7557,b' surface effects of a doped thin film made o...,b'the metal insulator transition ( mit ) based...,b'introduction\nmodel and method\nresults\ncon...
5488,b' we have implemented a simple digital system...,"b""laser cooling and multiphoton spectroscopy o...",b'introduction\nexperiment\nsystem performance'
2240,b' we analyze the electronic structure of atom...,b'in order to understand the quantum systems f...,b'introduction\ntheory and calculation methods...
9645,b' this proceeding explores some of the questi...,b'despite the wide difference in energy scales...,b'introduction\nthe @xmath0sm\nwhat can neutri...


In [23]:
df_train[ :1]

,abstract,article,section_names
179,"b' counting the frequencies of 3- , 4- , and 5...",b'design tools for counting the frequencies of...,b'introduction\nproblem formulation\nprelimina...


In [7]:
train_dataset

Dataset({
    features: ['abstract', 'article', 'section_names', '__index_level_0__'],
    num_rows: 9000
})

In [8]:
train_dataset["abstract"][0]

"b' counting the frequencies of 3- , 4- , and 5-node undirected motifs ( also know as graphlets ) is widely used for understanding complex networks such as social and biology networks . however , it is a great challenge to compute these metrics for a large graph due to the intensive computation . despite recent efforts to count triangles ( i.e. , 3-node undirected motif counting ) , \\n little attention has been given to developing scalable tools that can be used to characterize 4- and 5-node motifs . \\n in this paper , we develop computational efficient methods to sample and count 4- and 5- node undirected motifs . \\n our methods provide unbiased estimators of motif frequencies , and we derive simple and exact formulas for the variances of the estimators . \\n moreover , our methods are designed to fit vertex centric programming models , so they can be easily applied to current graph computing systems such as pregel and graphlab . \\n we conduct experiments on a variety of real - wo

In [32]:
# train_dataset[0]

The metric is an instance of datasets.Metric:

In [10]:
import datasets
from datasets import load_dataset, load_metric
metric = load_metric("rouge")
metric

<ipython-input-10-457110cf77f6>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

In [11]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

## Preprocessing the data

In [29]:
max_input_length = 8192
max_target_length = 512
batch_size = 2

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "t5-small"    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
tokenizer("Hello, this one sentence!")

{'input_ids': [8774, 6, 48, 80, 7142, 55, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [17]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [18]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [33]:
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["abstract"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [34]:
preprocess_function(train_dataset[0])

{'input_ids': [[21603, 10, 3, 115, 1], [21603, 10, 3, 31, 1], [21603, 10, 3, 26, 1], [21603, 10, 3, 15, 1], [21603, 10, 3, 7, 1], [21603, 10, 3, 23, 1], [21603, 10, 3, 122, 1], [21603, 10, 3, 29, 1], [21603, 10, 1], [21603, 10, 3, 17, 1], [21603, 10, 3, 32, 1], [21603, 10, 3, 32, 1], [21603, 10, 3, 40, 1], [21603, 10, 3, 7, 1], [21603, 10, 1], [21603, 10, 3, 89, 1], [21603, 10, 3, 32, 1], [21603, 10, 3, 52, 1], [21603, 10, 1], [21603, 10, 3, 75, 1], [21603, 10, 3, 32, 1], [21603, 10, 3, 76, 1], [21603, 10, 3, 29, 1], [21603, 10, 3, 17, 1], [21603, 10, 3, 23, 1], [21603, 10, 3, 29, 1], [21603, 10, 3, 122, 1], [21603, 10, 1], [21603, 10, 3, 17, 1], [21603, 10, 3, 107, 1], [21603, 10, 3, 15, 1], [21603, 10, 1], [21603, 10, 3, 89, 1], [21603, 10, 3, 52, 1], [21603, 10, 3, 15, 1], [21603, 10, 3, 1824, 1], [21603, 10, 3, 76, 1], [21603, 10, 3, 15, 1], [21603, 10, 3, 29, 1], [21603, 10, 3, 75, 1], [21603, 10, 3, 23, 1], [21603, 10, 3, 15, 1], [21603, 10, 3, 7, 1], [21603, 10, 1], [21603, 10, 

In [35]:
tokenized_train_data = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

In [41]:
tokenized_val_data = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


## Fine-tuning the model

In [37]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [38]:
batch_size = 2
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-STEMsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [39]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [40]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [45]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset= tokenized_train_data,
    eval_dataset=tokenized_val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/gloriaguo1986/t5-small-finetuned-STEMsum into local empty directory.


In [46]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: ignored

In [ ]:
trainer.push_to_hub()